# Cygnus A

This notebook is designed to work through the homework problem on Cygnus A.

In [ ]:
from pylab import *
from scipy.optimize import curve_fit
from scipy.interpolate import UnivariateSpline
from scipy.integrate import trapz, simps

Here is the data in array format.

In [ ]:
log_nu = array([7.0, 7.3, 7.7, 8.0, 8.3, 8.7, 9.0, 9.3, 9.7, 10.0])
log_Fnu = array([-18.88, -18.55, -18.67, -18.86, -19.09, -19.38, -19.63, -19.96, -20.43, -20.79])

We can make a scatter plot of the data, similar to what we did with Hubble's data set.

In [ ]:
figure(1,figsize=(8,8))
scatter(log_nu, log_Fnu, s=100, marker='s', label='Data')
#grid();
xticks(fontsize=20)
yticks(fontsize=20)
xlabel(r'$\log(\nu~{\rm[Hz]})$',fontsize=20)
ylabel(r'$\log( F_{\nu}~{\rm[erg\,s^{-1}\,cm^{-2}\,Hz^{-1}]})$',fontsize=20)
legend(loc='lower left',fontsize=20)
title('Cygnus A',fontsize=20);


## Power-law Exponent, $\alpha$

The first part of the problem is to find the power-law exponent $\alpha$ around $\log \nu = 8$.  We can do that very simply by calculating the slope between two points. Since $\log \nu = 8$ is one of the data points, we can actually do it from above and below, and take the average.

In [ ]:
log_nu[3]

Recall that the slope of a line (segment) is calculated by ''rise over run'',

${\rm slope} = \frac{y_2-y_1}{x_2 - x_1}$

Below we do just that, calculate the slope from one side and the other and take their average.  In this case the $\log F_{\nu}$ data is our 'y' and the $\log \nu$ is our 'x'.  We'll use two adjancent segments to calculate the slope:

In [ ]:
slope32 = (log_Fnu[3]-log_Fnu[2])/(log_nu[3]-log_nu[2])
slope43 = (log_Fnu[4]-log_Fnu[3])/(log_nu[4]-log_nu[3])
print('The average slope about the point of interest is {0:5.2f}.'.format(0.5*(slope32+slope43)))

But what if the situation was more complicated than this?  While there's certainly nothing wrong with the above, our data might be less uniformly distributed.  So let's use a more sophisticated approach!

We can fit a polynomial function (we don't even care exactly what the formula for it is) and then find the slope of that polynomial as follows.

Here we can use the Univartiate Spline class from SciPy.

In [ ]:
?UnivariateSpline

To use this class we need to give it the x- and y-data as well as the degree of the spline used to approximate the data.  The degree is given by `k`; if you omit this k argument, it will use a cubic spline to fit the data.  Here we'll try splines of degree 1, 2, and 3.

In [ ]:
f1=UnivariateSpline(x=log_nu[1:], y=log_Fnu[1:], k=1)
f2=UnivariateSpline(x=log_nu[1:], y=log_Fnu[1:], k=2)
f3=UnivariateSpline(x=log_nu[1:], y=log_Fnu[1:], k=3)

You'll notice, if you look closely, that I am ignoring the first data point (using the array index [1:]).  We're interested in the behavior of the data around $\log \nu = 8$ and that first data point will significantly influence the shape of our interpolating polynomial.

In [ ]:
figure(2,figsize=(8,8))
scatter(log_nu, log_Fnu, s=100, marker='s', color='Black', label='Data')

x=linspace(7.2,9.8,1000)
plot(x, f1(x), label='Linear', lw=2)
plot(x, f2(x), label='Quadratic', lw=2)
plot(x, f3(x), label='Cubic', lw=2, ls='--')

xticks(fontsize=20)
yticks(fontsize=20)
xlabel(r'$\log(\nu~{\rm[Hz]})$',fontsize=20)
ylabel(r'$\log( F_{\nu}~{\rm[erg\,s^{-1}\,cm^{-2}\,Hz^{-1}]})$',fontsize=20)

legend(loc='lower left',fontsize=20)
title('Cygnus A', fontsize=20);

We used the `UnivariateSpline` function here because it has a method attached to it that allows you to obtain the derivative at a specific point.  The  command below creates a new function that returns the derivative (slope) of the function at that point.  We then use the created function to evaluate the derivative at $\log \nu = 8$.:

In [ ]:
df1=f1.derivative()
df1(8.0).item()

In [ ]:
df2=f2.derivative()
df2(8.0).item()

In [ ]:
df3=f3.derivative()
df3(8.0).item()

Keep in mind that we write the equation $F_{\nu}=\nu^{-\alpha}$.

__Which spline deriviate do you think gives the best estimate of the power law exponent? Explain.__

## Integrate to find Total Luminosity

The second part of the question asks you to integrate $F_{\nu}$ to get the total luminosity.  This can be done in 2 parts.  The first is to actually integrate, which gives the total *flux* and then use the relationship between distance D, flux F, and luminosity L.  The simplest way to do that is to use a numerical integration method, like Simpson's Rule.  SciPy has a convenient version of it that we can use here.

In [ ]:
?simps

Here is an example where we integrate the function $y= x^2$ from x=0 to 5 using different methods.  (What's the analytical answer?)

Here we'll create an array of 11 equally spaced x-values from 0 to 5 and then make $y=x^2$.  After that it's simply a matter of giving the `simps` function these arrays to work on:

In [ ]:
x=linspace(0,5,11)
y=x*x
simps(y,x)

Note that this functions take $y$ as the first input and $x$ as the second.  So unless you give the names, e.g., `simps(x=x, y=y)`, you have to put $y$ first.

From this point you can use the same technique to calculate the integral $F=\int_a^b F_{\nu} d\nu$.  Note that you can obtain the linear (unlogged) versions of these arrays using the `pow()` function.

In [ ]:
?pow

In [ ]:
Fnu = pow(10,log_Fnu)
nu = pow(10,log_nu)

In [ ]:
simps(Fnu,nu)